In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import datetime
import os
import csv

In [2]:
# Pfad zur parquet Datei
file_path = "Data/AllStockDataDownload_XS.parquet"

# Parquet-Datei in DataFrame laden
all = pd.read_parquet(file_path)
all = all.rename(columns={'Adj Close': 'Value', 'ticker': 'Ticker'})

In [3]:
MovingAverage = [10, 20, 50, 100, 200, 500]

# Gruppiere die Daten nach Ticker
grouped = all.groupby('Ticker')

# Erstellen Sie ein leeres DataFrame, um die Ergebnisse zu speichern
result_df = pd.DataFrame()

for ticker, group in grouped:
    group_copy = group.copy()  # Erstellen Sie eine Kopie des Gruppen-DataFrames
    for w in MovingAverage:
        col_name = f'MA_{w}'
        group_copy[col_name] = group_copy['Value'].rolling(w).mean()
    result_df = pd.concat([result_df, group_copy])  # Füge die Gruppe dem resultierenden DataFrame hinzu

all = result_df.reset_index(drop=True)  # Setze den Index zurück, um die ursprüngliche Indexierung beizubehalten

all = all.astype({'MA_10': 'float32', 'MA_20': 'float32', 'MA_50': 'float32', 'MA_100': 'float32', 'MA_200': 'float32', 'MA_500': 'float32'})

In [4]:
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7672724 entries, 0 to 7672723
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Date    object 
 1   Value   float32
 2   Volume  float32
 3   Ticker  object 
 4   MA_10   float32
 5   MA_20   float32
 6   MA_50   float32
 7   MA_100  float32
 8   MA_200  float32
 9   MA_500  float32
dtypes: float32(8), object(2)
memory usage: 351.2+ MB


In [5]:
folder = 'Data'
filename = 'AllStockDataDownload_MAVG.csv'
file_path = os.path.join(folder, filename)
%time all.to_csv(file_path, index=False)

CPU times: total: 37.8 s
Wall time: 42.6 s


In [6]:
all['Ticker'].nunique()

1113